In [16]:
%load_ext autoreload
%autoreload 3

import gconcord as cc
import gconcorde as cce
import numpy as np

# import importlib
# importlib.reload(cc)

np.random.seed(3)

# true precision matrix
omega = np.zeros((3,3), order="C")
omega[0, 1] = 2.1
omega[1, 2] = -2.1
omega += omega.T
np.fill_diagonal(omega, 3)

# true covariance matrix
sigma = np.linalg.inv(omega)

# observations
data = np.random.multivariate_normal([0]*3, sigma, 500)

# sample covariance matrix
S = np.cov(data, rowvar=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
def get_precision(X):
    return .5*(np.diag(np.diag(X)) @ X) + .5*(X.T @ np.diag(np.diag(X)))

In [10]:
omega_hat, hist = cce.pycceista(S, 0.5, maxitr=1000)
print("omega_hat:\n %s\n" % (str(get_precision(omega_hat))))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist[-3:])))
print("iterations: %d" % (len(hist)))
print("successive norm diff: %e" % (hist[-1,1]))
print("objective: %e" % (hist[-1,2]))

omega_hat:
 [[ 1.16791916  0.57351906  0.15530139]
 [ 0.57351906  0.78436955 -0.57275944]
 [ 0.15530139 -0.57275944  1.17044823]]

hist: (inner_iter_count, delta_subg, objective)
 [[1.50000000e+01 3.69909808e-05 1.92169363e+00]
 [1.60000000e+01 1.84777373e-05 1.92168244e+00]
 [1.70000000e+01 9.23443516e-06 1.92167685e+00]]

iterations: 21
successive norm diff: 9.234435e-06
objective: 1.921677e+00


In [32]:
omega_hat, hist = cce.pycce_constant(S, 0.5, maxitr=1000)
print("omega_hat:\n %s\n" % (str(get_precision(omega_hat))))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(hist[-3:])))
print("iterations: %d" % (len(hist)))
print("successive norm diff: %e" % (hist[-1,1]))
print("objective: %e" % (hist[-1,2]))

omega_hat:
 [[ 1.57867827  0.86169305  0.        ]
 [ 0.86169305  0.97017702 -0.86123329]
 [ 0.         -0.86123329  1.58270433]]

hist: (inner_iter_count, delta_subg, objective)
 [[3.91000000e+02 1.03660229e-05 1.81275934e+00]
 [3.92000000e+02 1.01655514e-05 1.81275934e+00]
 [3.93000000e+02 9.96895937e-06 1.81275933e+00]]

iterations: 393
successive norm diff: 9.968959e-06
objective: 1.812759e+00


In [3]:
S = np.cov(Xs[0], rowvar=False)
start_time = time.time()
Omega_hat, hist = cce.pycce_constant(S, 0.03, epstol=1e-5, maxitr=1000, penalize_diagonal=False)
time_took = time.time() - start_time
Omega_hat_scaled= .5*(np.diag(np.diag(Omega_hat)) @ Omega_hat) + .5*(Omega_hat.T @ np.diag(np.diag(Omega_hat)))
print('cce-constant')
# print(' ')
# print(Omega_hat_scaled)
print(' ')
print('# of iterations:', len(hist))
print('Objective:', hist[-1][2])
print('Time:', time_took)
print('MCC:', mcc(Omega, Omega_hat_scaled))
print(' ')
S = np.cov(Xs[0], rowvar=False)
start_time = time.time()
Omega_hat, hist = cce.pycceista(S, 0.03, epstol=1e-5, maxitr=1000, penalize_diagonal=False)
time_took = time.time() - start_time
Omega_hat_scaled= .5*(np.diag(np.diag(Omega_hat)) @ Omega_hat) + .5*(Omega_hat.T @ np.diag(np.diag(Omega_hat)))
print('cce-ista')
# print(' ')
# print(Omega_hat_scaled)
print(' ')
print('# of outer iterations:', len(hist))
count = 0
for i in range(len(hist)):
    count += hist[i][0]
print('# of total inner iterations:', int(count))
print('Objective:', hist[-1][2])
print('Time:', time_took)
print('MCC:', mcc(Omega, Omega_hat_scaled))
print(' ')

cce-constant
 
# of iterations: 19
Objective: 149.15794986582847
Time: 0.12027239799499512
MCC: 0.22791347494840594
 
cce-ista
 
# of outer iterations: 10
# of total inner iterations: 28
Objective: 149.15794987167354
Time: 0.4472823143005371
MCC: 0.22791347494840594
 
